### 第9回レポート課題その1

先のものに修正を加えました.

In [2]:
!pip install gensim

In [4]:
!wget http://130.153.158.5/~inaba/GoogleNews-vectors-negative300.bin

--2020-08-02 21:10:02--  http://130.153.158.5/~inaba/GoogleNews-vectors-negative300.bin
Connecting to 130.153.158.5:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3644258522 (3.4G) [application/octet-stream]
Saving to: ‘GoogleNews-vectors-negative300.bin’

GoogleNews-vectors- 100%[===================>]   3.39G  29.3MB/s    in 1m 59s  

2020-08-02 21:12:01 (29.2 MB/s) - ‘GoogleNews-vectors-negative300.bin’ saved [3644258522/3644258522]



In [5]:
!wget http://130.153.158.5/~inaba/data70.zip

--2020-08-02 21:12:07--  http://130.153.158.5/~inaba/data70.zip
Connecting to 130.153.158.5:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 392010 (383K) [application/zip]
Saving to: ‘data70.zip’

data70.zip          100%[===================>] 382.82K  1.30KB/s    in 6.4s    

2020-08-02 21:12:13 (59.6 KB/s) - ‘data70.zip’ saved [392010/392010]



In [6]:
!unzip data70.zip

Archive:  data70.zip
  inflating: test.txt                
  inflating: train.txt               
  inflating: valid.txt               


In [8]:
import gensim
import re
import torch
import numpy as np

w2v_path = './GoogleNews-vectors-negative300.bin' #todo GoogleNews-vectors-negative300.binのパスを指定
data_path = './' #todo ファイル出力ディレクトリを指定
data_path2 = '/content/drive/My Drive/data/'

w2v_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_path, binary=True) #todo gensimでword2vecのモデルを読み込む

category2num = {"b": 0, "t": 1, "e": 2, "m": 3}


def get_feature(title):
    word_list = title.split(' ')#todo タイトルをスペースで分割
    vec_list = []
    for word in word_list:
        try:
            vec = w2v_model[word]#todo wordを意味するベクトルを取得
        except KeyError:
            vec = np.zeros(300)#todo すべての要素が0のベクトルを代入
        vec_list.append(vec)    
    vec_np = np.array(vec_list) # numpyのarrayに変換
    feature = np.sum(vec_np, axis=0)/len(vec_np)# 平均ベクトルを計算
    return feature


def get_data(fname):
    label_list = []
    feature_list = []
    with open(fname, encoding="utf8") as f:
        for line in f:
            if not line:
                continue
            data = line.split('\t')
            title = data[1]#todo dataからタイトルを取り出す
            feature = get_feature(title) 
            feature_list.append(feature)
            label = category2num[data[0]]
            label_list.append(label)

    features = torch.tensor(feature_list)#todo feature_listをtensorに変換
    labels = torch.tensor(label_list)#todo label_listをtensorに変換
    return features, labels


train_x, train_y = get_data(data_path + "train.txt")
valid_x, valid_y = get_data(data_path + "valid.txt")
test_x, test_y = get_data(data_path + "test.txt")

# 保存
torch.save(train_x, data_path2 + "train_x.pt")
torch.save(train_y, data_path2 + "train_y.pt")
torch.save(valid_x, data_path2 + "valid_x.pt")
torch.save(valid_y, data_path2 + "valid_y.pt")
torch.save(test_x, data_path2 + "test_x.pt")
torch.save(test_y, data_path2 + "test_y.pt")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import time
from google.colab import drive
drive.mount('/content/drive')

# 単層ニューラルネットワークを定義
class SingleLayerNN(nn.Module):
    def __init__(self, embedding_dim, num_labels):
        super(SingleLayerNN, self).__init__()
        self.linear = nn.Linear(embedding_dim, num_labels) #todo linear層を追加
    
    def forward(self, x):
        h1 = self.linear(x) #todo xをlinear層に入力
        softmax = nn.Softmax()
        return softmax(h1) #todo h1をsoftmaxしたものを返す

class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x) #todo データサイズを返す

    def __getitem__(self, idx):
        out_x = self.x[idx] #todo idx番目のxの要素を返す
        out_y = self.y[idx] #todo idx番目のyの要素を返す
        return out_x, out_y
        
torch.manual_seed(1)

embedding_dim = 300
num_labels = 4
NUM_EPOCH = 100

train_x, train_y = torch.load("/content/drive/My Drive/data/train_x.pt"), torch.load("/content/drive/My Drive/data/train_y.pt")
valid_x, valid_y = torch.load("/content/drive/My Drive/data/valid_x.pt"), torch.load("/content/drive/My Drive/data/valid_y.pt")

# データとラベルを1つにまとめる
dataset = MyDataset(train_x, train_y)

loss_fn = nn.CrossEntropyLoss()#todo 損失関数としてCrossEntropyLossを用いる


def train(model, train_loader):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        pred = model.forward(data.type(torch.FloatTensor))  #todo dataをmodelに入力
        loss = loss_fn(pred, target)  #todo predとtargetと損失関数からロスを計算
        loss.backward() # 逆誤差伝搬を実施
        optimizer.step() # パラメータを更新

        
def evaluation(model, data, target):
    model.eval()
    with torch.no_grad():
        pred =  model(data.type(torch.FloatTensor)) #todo dataをmodelに入力
        loss = loss_fn(pred, target)  #todo predとtargetと損失関数からロスを計算
        pred_labels = torch.argmax(pred, axis=1)  #todo 推定したラベルを代入
        acc = (pred_labels == target).sum().item() / len(pred_labels) #todo pred_labelsとtargetから正解率を計算
    return loss.item(), acc


batch_size_list = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024] #todo バッチサイズの値のリスト．1,2,4,8,…が入る

for batch_size in batch_size_list:
    model = SingleLayerNN(embedding_dim, num_labels)
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    # ミニバッチを扱うためのデータローダを作成
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(NUM_EPOCH):
        start_time = time.time() # 学習開始時の時間を取得
        train(model, train_loader)
        took_time = time.time() - start_time #todo 学習にかかった時間を代入

        valid_loss, valid_acc = evaluation(model, valid_x, valid_y)

        if epoch % 10 == 0:
            print(f"batch size:{batch_size}\tepoch: {epoch}")
            print(f"train time: {took_time}")
            print(f"<valid>\tloss: {valid_loss}\tacc: {valid_acc}")
    
    print("\n================\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


batch size:1	epoch: 0
train time: 3.2745683193206787
<valid>	loss: 0.9587206244468689	acc: 0.800599700149925
batch size:1	epoch: 10
train time: 3.1874783039093018
<valid>	loss: 0.9397221803665161	acc: 0.8013493253373314
batch size:1	epoch: 20
train time: 3.032317638397217
<valid>	loss: 0.9378631114959717	acc: 0.8020989505247377
batch size:1	epoch: 30
train time: 3.1171605587005615
<valid>	loss: 0.936996340751648	acc: 0.8035982008995503
batch size:1	epoch: 40
train time: 3.2007553577423096
<valid>	loss: 0.9364750981330872	acc: 0.802848575712144
batch size:1	epoch: 50
train time: 3.1456024646759033
<valid>	loss: 0.9360454082489014	acc: 0.8020989505247377
batch size:1	epoch: 60
train time: 3.2307629585266113
<valid>	loss: 0.8930366635322571	acc: 0.8538230884557722
batch size:1	epoch: 70
train time: 3.5577666759490967
<valid>	loss: 0.8867972493171692	acc: 0.8590704647676162
batch size:1	epoch: 80
train time: 3.170229196548462
<valid>	loss: 0.885372519493103	acc: 0.8598200899550225
batch si

バッチサイズが大きくなるほど計算にかかる時間が短くなっていることが分かる.

#### 感想
穴埋めはPytorchの復習にもなり, 前回よりもやりやすかった. ただ細かいエラーの対処に時間がかかったので今後の課題だと感じた.